In [ ]:
from sklearn.model_selection import train_test_split
from pandas import DataFrame, read_csv, concat, get_dummies, Series, CategoricalDtype
from sklearn import metrics
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.metrics import mean_squared_log_error, mean_squared_error
from sklearn.preprocessing import normalize, StandardScaler, LabelEncoder, OneHotEncoder, OrdinalEncoder, FunctionTransformer
from sklearn.ensemble import IsolationForest
from sklearn.manifold import TSNE
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator
import xgboost
from xgboost import plot_importance, XGBRegressor
from pprint import pprint
from json import load
import seaborn as sns
import numpy as np
from math import sqrt

%matplotlib inline

In [ ]:
project = 'house-prices'
version='v0.1'

In [ ]:
fulltrain=read_csv('./train.csv',index_col=0)
test=read_csv('./test.csv',index_col=0)
fixed_seed=1234578416
train80, valid20 = train_test_split(fulltrain, test_size=0.2, random_state=fixed_seed)

In [ ]:
target_column='SalePrice'
X_train = train80.drop(target_column, axis=1)
y_train = train80[target_column]
X_val = valid20.drop(target_column, axis=1)
y_val = valid20[target_column]

In [ ]:
categories = load(open('categories.json',"r"))

In [ ]:
num_columns = [c for c in X_train.columns if c not in categories.keys()]

In [ ]:
ordinals={}
for key, value in list(categories.items()):
    if value[0] == 'Ex':
        ordinals[key] = value
        categories.pop(key)
for col, tags in ordinals.items():
    tags.reverse()
    
ord_columns = list(ordinals.keys())
ord_values = list(ordinals.values())
cat_columns = list(categories.keys())
cat_values = list(categories.keys())

In [ ]:
ordinals.values()

In [ ]:
# for col in ord_columns:
#     print(X_train[col].unique())
# for col in cat_columns:
#     print(X_train[col].unique())

Ordinal Encoding

In [ ]:
ordinal_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder(categories=ord_values))
])

In [ ]:
categorical_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
#    ('imputer', SimpleImputer(strategy='constant', fill_value='NA')),
    ('onehot', OneHotEncoder(handle_unknown="ignore"))
])

In [ ]:
numeric_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

In [ ]:
preprocess_pipe = ColumnTransformer([
    ('cat', categorical_pipe, cat_columns),
    ('num', numeric_pipe, num_columns),
    ('ord', ordinal_pipe, ord_columns)
])

## Removing outliers

In [ ]:
outlier_detection = Pipeline([
    ('pp', preprocess_pipe),
    ('outlier', IsolationForest(max_samples=100, random_state=42))
])

In [ ]:
outliers = outlier_detection.fit_predict(X_train)

X_train_clean = X_train.loc[outliers==1,:]
y_train_clean = y_train.loc[outliers==1]

## Preprocess Train, Test and Valid sets

In [ ]:
X_train_process = preprocess_pipe.fit_transform(X_train_clean)
test_process = preprocess_pipe.transform(test)
X_valid_process = preprocess_pipe.transform(X_val)

## Output processing

In [ ]:
output_pipe = Pipeline([
    ('log', FunctionTransformer(np.log)),
    ('scaler', StandardScaler())
])

In [ ]:
output_pipe.fit(y_train.to_numpy().reshape(-1,1))
y_train_process = output_pipe.transform(y_train.to_numpy().reshape(-1,1))
y_val_process = output_pipe.transform(y_val.to_numpy().reshape(-1,1))

Surface Area

In [ ]:
class AddSurface(TransformerMixin, BaseEstimator):
    

In [ ]:
for df in [X_train, X_val, test]:
    df['Surface'] =  df['2ndFlrSF'] + df['1stFlrSF'] + df['TotalBsmtSF']
    df.drop('GrLivArea', axis=1, inplace=True)

In [ ]:
for df in [X_train, X_val, test]:
    na_sum = df.isna().sum()
    print(na_sum[na_sum!=0])

# Outliers detection

In [ ]:
clf = IsolationForest(max_samples=100, random_state=42, behaviour='new')
clf.fit(X_train)
y_pred_train = DataFrame(data=clf.predict(X_train),index=X_train.index)
y_pred_train_str = ('C' + y_pred_train.astype('str'))[0]
tsne = TSNE(n_components=2)
X_tsne = tsne.fit_transform(X_train)
plt.figure(figsize=(16,12))
ax =sns.scatterplot(x=X_tsne[:,0], y=X_tsne[:,1], hue=y_pred_train_str)

Removing outliers

In [ ]:
isnotoutlier = (y_pred_train[0]==1)
X_train = X_train.loc[isnotoutlier,:]
y_train = y_train.loc[isnotoutlier]
nb_outliers = (y_pred_train[0]==-1).sum()
print('There was ' + str(nb_outliers) + ' outliers')

In [ ]:
zero_cols=X_train.columns[( X_train == 0).all()]
display(zero_cols)

In [ ]:
for df in [X_train, X_val, test]:
    df.drop(columns=zero_cols, inplace=True)

In [ ]:
# normalization
scaler = StandardScaler()
scaler.fit(X_train)

X_train[:] = scaler.transform(X_train)
X_val[:] = scaler.transform(X_val)
test[:] = scaler.transform(test)

Removing empty columns

TODO remove constant columns

In [ ]:
zero_cols = X_train.columns[( X_train == 0).all()]
display(zero_cols)
for df in [X_train, X_val, test]:
    df.drop(columns=zero_cols, inplace=True)
X_train.shape

Gridsearch hyperparameters estimation

In [ ]:
# from hypopt import GridSearch
# params = {'min_child_weight':[6,7,8,9], 'gamma':[i/100.0 for i in range(1,5)],  'subsample':[i/10.0 for i in range(2,5)],
# 'colsample_bytree':[i/10.0 for i in range(8,10)], 'max_depth': [3,4,5]}

# model = XGBRegressor(booster="gbtree")
# grid = GridSearch(model, params)
# grid.fit(X_train, y_train, X_val, y_val)
# grid.best_params

XGBoost model

In [ ]:
model = XGBRegressor(booster="gbtree",colsample_bytree=0.9,
                     max_depth=4, n_estimators=400, gamma= 0.01,
                     min_child_weight=8,
                     subsample=0.3)

In [ ]:
# model = XGBRegressor(booster="gbtree",colsample_bytree=0.5,
#                      max_depth=3, n_estimators=400, subsample=0.7)

In [ ]:
model.fit(X_train,y_train)

In [ ]:
y_pred = model.predict(X_val)

In [ ]:
RMSLE = sqrt(mean_squared_error(y_val,y_pred))
RMSLE

In [ ]:
test_pred = model.predict(test)

In [ ]:
submission = DataFrame({"SalePrice": np.exp (test_pred)}, index=test.index)
submission.to_csv('test-prediction.csv')

In [ ]:
sns.distplot(y_pred-y_val)
plt.show()

In [ ]:
y_t_pred = model.predict(X_train)
print(sqrt(mean_squared_error(y_train,y_t_pred)))
sns.distplot(y_t_pred-y_train)
plt.show()

# Feature importance
Feature importance as reported by XGBoost

In [ ]:
importance_dict = model.get_booster().get_score(importance_type="gain")
importance = DataFrame.from_dict(importance_dict, orient='index')[0].sort_values(ascending=False)
order = list(importance.index)

In [ ]:
nb_bars = 25

plt.figure(figsize=(18,20))
plt.subplots_adjust(hspace=0.5)
for i in range(0,2):
    plt.subplot(4,1,i+1)
    low = nb_bars*i
    hi = nb_bars*(i+1)
    bars = sns.barplot(x=importance[low:hi].index, y=importance[low:hi])
    bars.set_xticklabels(bars.get_xticklabels(), rotation=45)
plt.show()

Reordering features by importance

In [ ]:
X_train = X_train[order]
X_val = X_val[order]
test = test[order]
X_train.shape

Save prepared datasets

In [ ]:
X_train.to_csv('X_train.csv')
X_val.to_csv('X_val.csv')
test.to_csv('X_test.csv')
y_train.to_csv('y_train.csv', header=True)
y_val.to_csv('y_val.csv', header=True)